# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 20 2022 10:40AM,246711,15,30010788,"Alliance Pharma, Inc.",Released
1,Sep 20 2022 10:40AM,246711,15,30010789,"Alliance Pharma, Inc.",Released
2,Sep 20 2022 10:40AM,246711,15,30010791,"Alliance Pharma, Inc.",Released
3,Sep 20 2022 10:40AM,246711,15,30010790,"Alliance Pharma, Inc.",Released
4,Sep 20 2022 10:40AM,246711,15,30010792,"Alliance Pharma, Inc.",Released
5,Sep 20 2022 10:40AM,246711,15,30010793,"Alliance Pharma, Inc.",Released
6,Sep 20 2022 10:40AM,246711,15,30010795,"Alliance Pharma, Inc.",Released
7,Sep 20 2022 10:40AM,246711,15,30010794,"Alliance Pharma, Inc.",Released
8,Sep 20 2022 10:40AM,246711,15,30010797,"Alliance Pharma, Inc.",Released
9,Sep 20 2022 10:40AM,246711,15,30010798,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,246806,Executing,1
26,246806,Released,23
27,246807,Released,1
28,246809,Released,16
29,246810,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246801,NaN,NaN,9.0
246806,NaN,1.0,23.0
246807,NaN,NaN,1.0
246809,NaN,NaN,16.0
246810,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0.0,0.0,32.0
246712,52.0,0.0,12.0
246716,0.0,1.0,0.0
246724,0.0,0.0,1.0
246762,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0,0,32
246712,52,0,12
246716,0,1,0
246724,0,0,1
246762,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,0,0,32
1,246712,52,0,12
2,246716,0,1,0
3,246724,0,0,1
4,246762,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,,,32
1,246712,52,,12
2,246716,,1,
3,246724,,,1
4,246762,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 20 2022 10:40AM,246711,15,"Alliance Pharma, Inc."
32,Sep 20 2022 10:39AM,246801,19,Eywa Pharma Inc.
41,Sep 20 2022 10:15AM,246810,10,"ClearSpec, LLC"
42,Sep 20 2022 10:13AM,246809,10,Eywa Pharma Inc.
58,Sep 20 2022 10:01AM,246771,21,"NBTY Global, Inc."
59,Sep 20 2022 9:56AM,246786,10,ISDIN Corporation
90,Sep 20 2022 9:54AM,246770,21,"NBTY Global, Inc."
91,Sep 20 2022 9:54AM,246807,10,"CLINUVEL, Inc."
92,Sep 20 2022 9:52AM,246806,20,"ACI Healthcare USA, Inc."
116,Sep 20 2022 9:32AM,246799,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 20 2022 10:40AM,246711,15,"Alliance Pharma, Inc.",,,32
1,Sep 20 2022 10:39AM,246801,19,Eywa Pharma Inc.,,,9
2,Sep 20 2022 10:15AM,246810,10,"ClearSpec, LLC",,,1
3,Sep 20 2022 10:13AM,246809,10,Eywa Pharma Inc.,,,16
4,Sep 20 2022 10:01AM,246771,21,"NBTY Global, Inc.",,1,
5,Sep 20 2022 9:56AM,246786,10,ISDIN Corporation,,,31
6,Sep 20 2022 9:54AM,246770,21,"NBTY Global, Inc.",,1,
7,Sep 20 2022 9:54AM,246807,10,"CLINUVEL, Inc.",,,1
8,Sep 20 2022 9:52AM,246806,20,"ACI Healthcare USA, Inc.",,1,23
9,Sep 20 2022 9:32AM,246799,10,"Methapharm, Inc.",,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 10:40AM,246711,15,"Alliance Pharma, Inc.",32,,
1,Sep 20 2022 10:39AM,246801,19,Eywa Pharma Inc.,9,,
2,Sep 20 2022 10:15AM,246810,10,"ClearSpec, LLC",1,,
3,Sep 20 2022 10:13AM,246809,10,Eywa Pharma Inc.,16,,
4,Sep 20 2022 10:01AM,246771,21,"NBTY Global, Inc.",,1,
5,Sep 20 2022 9:56AM,246786,10,ISDIN Corporation,31,,
6,Sep 20 2022 9:54AM,246770,21,"NBTY Global, Inc.",,1,
7,Sep 20 2022 9:54AM,246807,10,"CLINUVEL, Inc.",1,,
8,Sep 20 2022 9:52AM,246806,20,"ACI Healthcare USA, Inc.",23,1,
9,Sep 20 2022 9:32AM,246799,10,"Methapharm, Inc.",7,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 10:40AM,246711,15,"Alliance Pharma, Inc.",32,,
1,Sep 20 2022 10:39AM,246801,19,Eywa Pharma Inc.,9,,
2,Sep 20 2022 10:15AM,246810,10,"ClearSpec, LLC",1,,
3,Sep 20 2022 10:13AM,246809,10,Eywa Pharma Inc.,16,,
4,Sep 20 2022 10:01AM,246771,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 10:40AM,246711,15,"Alliance Pharma, Inc.",32.0,NaN,NaN
1,Sep 20 2022 10:39AM,246801,19,Eywa Pharma Inc.,9.0,NaN,NaN
2,Sep 20 2022 10:15AM,246810,10,"ClearSpec, LLC",1.0,NaN,NaN
3,Sep 20 2022 10:13AM,246809,10,Eywa Pharma Inc.,16.0,NaN,NaN
4,Sep 20 2022 10:01AM,246771,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 10:40AM,246711,15,"Alliance Pharma, Inc.",32.0,0.0,0.0
1,Sep 20 2022 10:39AM,246801,19,Eywa Pharma Inc.,9.0,0.0,0.0
2,Sep 20 2022 10:15AM,246810,10,"ClearSpec, LLC",1.0,0.0,0.0
3,Sep 20 2022 10:13AM,246809,10,Eywa Pharma Inc.,16.0,0.0,0.0
4,Sep 20 2022 10:01AM,246771,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3208319,96.0,15.0,0.0
15,493423,44.0,0.0,52.0
16,1233809,5.0,0.0,0.0
19,246801,9.0,0.0,0.0
20,493591,23.0,2.0,0.0
21,740257,0.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3208319,96.0,15.0,0.0
1,15,493423,44.0,0.0,52.0
2,16,1233809,5.0,0.0,0.0
3,19,246801,9.0,0.0,0.0
4,20,493591,23.0,2.0,0.0
5,21,740257,0.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,96.0,15.0,0.0
1,15,44.0,0.0,52.0
2,16,5.0,0.0,0.0
3,19,9.0,0.0,0.0
4,20,23.0,2.0,0.0
5,21,0.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96.0
1,15,Released,44.0
2,16,Released,5.0
3,19,Released,9.0
4,20,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,52.0,0.0,0.0,0.0,0.0
Executing,15.0,0.0,0.0,0.0,2.0,3.0
Released,96.0,44.0,5.0,9.0,23.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,52.0,0.0,0.0,0.0,0.0
1,Executing,15.0,0.0,0.0,0.0,2.0,3.0
2,Released,96.0,44.0,5.0,9.0,23.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,52.0,0.0,0.0,0.0,0.0
1,Executing,15.0,0.0,0.0,0.0,2.0,3.0
2,Released,96.0,44.0,5.0,9.0,23.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()